In [1]:
#r "nuget:Docker.DotNet"
#r "nuget:Npgsql"

using Docker.DotNet;
using Docker.DotNet.Models;
using Npgsql;
using System.Threading;
using System.Threading.Tasks;
using System.Data.Common;

Installing package Docker.DotNet................done!

Successfully added reference to package Docker.DotNet, version 3.125.2

Installing package Npgsql.....done!

Successfully added reference to package Npgsql, version 4.1.1

In [2]:
DbConnection Db;
DockerClient _client;
CreateContainerResponse _containerResponse;

_client = new DockerClientConfiguration(new Uri("npipe://./pipe/docker_engine")).CreateClient();

In [3]:
var hostPort = new Random((int)DateTime.UtcNow.Ticks).Next(10000, 12000);

var images = await _client.Images.ListImagesAsync(new ImagesListParameters()
    {
    MatchName = "postgres:latest",
    }, CancellationToken.None);

//check if container exists
    var pgImage = images.FirstOrDefault();
    if (pgImage == null)
        throw new Exception($"Docker image for postgres:latest not found.");

    //create container from image
    var container = await _client.Containers.CreateContainerAsync(new CreateContainerParameters()
    {
        User = "postgres",
        Env = new List<string>()
        {
            "POSTGRES_PASSWORD=password",
            "POSTGRES_DB=repotest",
            "POSTGRES_USER=postgres"
        },
        ExposedPorts = new Dictionary<string, EmptyStruct>()
        {
            ["5432"] = new EmptyStruct()
        },
        HostConfig = new HostConfig()
        {
            PortBindings = new Dictionary<string, IList<PortBinding>>()
            {
                ["5432"] = new List<PortBinding>()
                    {new PortBinding() {HostIP = "0.0.0.0", HostPort = $"{hostPort}"}}
            }
        },
        Image = "postgres:latest",
    }, CancellationToken.None);
    
    if (!await _client.Containers.StartContainerAsync(container.ID, new ContainerStartParameters()
    {
        DetachKeys = "d=postgres"
    }, CancellationToken.None))
    {
        throw new Exception($"Could not start container: {container.ID}");
    }

    var count = 10;
    Thread.Sleep(5000);
    var containerStat = await _client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);
    while (!containerStat.State.Running && count-- > 0)
    {
        Thread.Sleep(1000);
        containerStat = await _client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);

    }
    Thread.Sleep(10000); //I need some time for the DB to finish starting up so that my tests don't report the DB is starting up


In [4]:
var connectionStringBuilder = new NpgsqlConnectionStringBuilder() { ConnectionString = $"User ID=postgres;Password=password;Server=127.0.0.1;Port={hostPort};Database=repotest;Integrated Security=true;Pooling=false;CommandTimeout=300" };
Db = new NpgsqlConnection(connectionStringBuilder.ConnectionString);
Db.Open();

Console.WriteLine(String.Equals("repotest", Db.Database)); 

True


# Creating a new database with a new user, connecting with user

In [5]:
using (var cmd = Db.CreateCommand()) {
    cmd.CommandText = "CREATE DATABASE testdb";
    cmd.ExecuteNonQuery();
    
    cmd.CommandText = "CREATE USER pgtester CREATEROLE LOGIN NOREPLICATION PASSWORD 'abcdef@123'";
    cmd.ExecuteNonQuery();
    
    cmd.CommandText = "GRANT ALL PRIVILEGES ON DATABASE testdb TO pgtester;";
    cmd.ExecuteNonQuery();
}

Db.Close(); 
connectionStringBuilder = new NpgsqlConnectionStringBuilder() { ConnectionString = $"User ID=pgtester;Password=abcdef@123;Server=127.0.0.1;Port={hostPort};Database=testdb;Integrated Security=true;Pooling=false;CommandTimeout=300" };
Db = new NpgsqlConnection(connectionStringBuilder.ConnectionString);
Db.Open();

# Creating a table and filling it up 

In [6]:
using (var cmd = Db.CreateCommand()) {
    cmd.CommandText = "DROP TABLE IF EXISTS counterparties";
    cmd.ExecuteNonQuery();

    cmd.CommandText = "CREATE TABLE counterparties(id serial PRIMARY KEY, cp_name VARCHAR(100) NOT NULL, created_on TIMESTAMP NOT NULL)";
    cmd.ExecuteNonQuery();

    cmd.CommandText = "INSERT INTO counterparties(cp_name, created_on) VALUES ('Investment bank A', '2019-11-11')";
    cmd.ExecuteNonQuery();
    
    cmd.CommandText = "INSERT INTO counterparties(cp_name, created_on) VALUES ('Investment bank B','" + DateTime.Now.ToString("yyyy-MM-dd") + "')";
    cmd.ExecuteNonQuery();
}

# Using bound variables for the insertion

In [7]:
using (var npgCmd = new NpgsqlCommand("INSERT INTO counterparties(cp_name, created_on) VALUES (@n, @d)", (NpgsqlConnection)Db))
{
    npgCmd.Parameters.AddWithValue("n", "Energy Company A");
    npgCmd.Parameters.AddWithValue("d", DateTime.Now);
    npgCmd.ExecuteNonQuery();
}

# Checking how many rows were inserted

In [8]:
using (var cmd = Db.CreateCommand()) {
cmd.CommandText = "SELECT COUNT(*) FROM counterparties";
    var nRows = cmd.ExecuteScalar();
    Console.WriteLine($"Total rows in table - Expected 3 - Found {nRows}");
}

Total rows in table - Expected 3 - Found 3


In [9]:
Db.Close(); 
Db.Dispose();

In [10]:
if (await _client.Containers.StopContainerAsync(container.ID, new ContainerStopParameters(), CancellationToken.None))
{
    Console.WriteLine("Container " + container.ID + " Stopped.");
    //delete container
    await _client.Containers.RemoveContainerAsync(container.ID, new ContainerRemoveParameters(), CancellationToken.None);
    Console.WriteLine("Container " + container.ID + " Deleted.");
}
else
{
    Console.WriteLine("Container " + container.ID + " Unaffacted.");
}

_client?.Dispose();

Container 25cac0d21f91ea3de7a3e5eb42ddbbeb3fade6c6ffc352bac9080d319ea30167 Stopped.
Container 25cac0d21f91ea3de7a3e5eb42ddbbeb3fade6c6ffc352bac9080d319ea30167 Deleted.
